# Trabajo Práctico Final: Linear/Quadratic Discriminant Analysis (LDA/QDA)

### Definición: Clasificador Bayesiano

Sean $k$ poblaciones, $x \in \mathbb{R}^p$ puede pertenecer a cualquiera $g \in \mathcal{G}$ de ellas. Bajo un esquema bayesiano, se define entonces $\pi_j \doteq P(G = j)$ la probabilidad *a priori* de que $X$ pertenezca a la clase *j*, y se **asume conocida** la distribución condicional de cada observable dado su clase $f_j \doteq f_{X|G=j}$.

De esta manera dicha probabilidad *a posteriori* resulta
$$
P(G|_{X=x} = j) = \frac{f_{X|G=j}(x) \cdot p_G(j)}{f_X(x)} \propto f_j(x) \cdot \pi_j
$$

La regla de decisión de Bayes es entonces
$$
H(x) \doteq \arg \max_{g \in \mathcal{G}} \{ P(G|_{X=x} = j) \} = \arg \max_{g \in \mathcal{G}} \{ f_j(x) \cdot \pi_j \}
$$

es decir, se predice a $x$ como perteneciente a la población $j$ cuya probabilidad a posteriori es máxima.

*Ojo, a no desesperar! $\pi_j$ no es otra cosa que una constante prefijada, y $f_j$ es, en su esencia, un campo escalar de $x$ a simplemente evaluar.*

### Distribución condicional

Para los clasificadores de discriminante cuadrático y lineal (QDA/LDA) se asume que $X|_{G=j} \sim \mathcal{N}_p(\mu_j, \Sigma_j)$, es decir, se asume que cada población sigue una distribución normal.

Por definición, se tiene entonces que para una clase $j$:
$$
f_j(x) = \frac{1}{(2 \pi)^\frac{p}{2} \cdot |\Sigma_j|^\frac{1}{2}} e^{- \frac{1}{2}(x-\mu_j)^T \Sigma_j^{-1} (x- \mu_j)}
$$

Aplicando logaritmo (que al ser una función estrictamente creciente no afecta el cálculo de máximos/mínimos), queda algo mucho más práctico de trabajar:

$$
\log{f_j(x)} = -\frac{1}{2}\log |\Sigma_j| - \frac{1}{2} (x-\mu_j)^T \Sigma_j^{-1} (x- \mu_j) + C
$$

Observar que en este caso $C=-\frac{p}{2} \log(2\pi)$, pero no se tiene en cuenta ya que al tener una constante aditiva en todas las clases, no afecta al cálculo del máximo.

### LDA

En el caso de LDA se hace una suposición extra, que es $X|_{G=j} \sim \mathcal{N}_p(\mu_j, \Sigma)$, es decir que las poblaciones no sólo siguen una distribución normal sino que son de igual matriz de covarianzas. Reemplazando arriba se obtiene entonces:

$$
\log{f_j(x)} =  -\frac{1}{2}\log |\Sigma| - \frac{1}{2} (x-\mu_j)^T \Sigma^{-1} (x- \mu_j) + C
$$

Ahora, como $-\frac{1}{2}\log |\Sigma|$ es común a todas las clases se puede incorporar a la constante aditiva y, distribuyendo y reagrupando términos sobre $(x-\mu_j)^T \Sigma^{-1} (x- \mu_j)$ se obtiene finalmente:

$$
\log{f_j(x)} =  \mu_j^T \Sigma^{-1} (x- \frac{1}{2} \mu_j) + C'
$$

### Entrenamiento/Ajuste

Obsérvese que para ambos modelos, ajustarlos a los datos implica estimar los parámetros $(\mu_j, \Sigma_j) \; \forall j = 1, \dots, k$ en el caso de QDA, y $(\mu_j, \Sigma)$ para LDA.

Estos parámetros se estiman por máxima verosimilitud, de manera que los estimadores resultan:

* $\hat{\mu}_j = \bar{x}_j$ el promedio de los $x$ de la clase *j*
* $\hat{\Sigma}_j = s^2_j$ la matriz de covarianzas estimada para cada clase *j*
* $\hat{\pi}_j = f_{R_j} = \frac{n_j}{n}$ la frecuencia relativa de la clase *j* en la muestra
* $\hat{\Sigma} = \frac{1}{n} \sum_{j=1}^k n_j \cdot s^2_j$ el promedio ponderado (por frecs. relativas) de las matrices de covarianzas de todas las clases. *Observar que se utiliza el estimador de MV y no el insesgado*

Es importante notar que si bien todos los $\mu, \Sigma$ deben ser estimados, la distribución *a priori* puede no inferirse de los datos sino asumirse previamente, utilizándose como entrada del modelo.

### Predicción

Para estos modelos, al igual que para cualquier clasificador Bayesiano del tipo antes visto, la estimación de la clase es por método *plug-in* sobre la regla de decisión $H(x)$, es decir devolver la clase que maximiza $\hat{f}_j(x) \cdot \hat{\pi}_j$, o lo que es lo mismo $\log\hat{f}_j(x) + \log\hat{\pi}_j$.

## Estructura del código

## Modelo 

In [1]:
import numpy as np
from numpy.linalg import det, inv

In [2]:
class ClassEncoder:
  def fit(self, y):
    self.names = np.unique(y)
    self.name_to_class = {name:idx for idx, name in enumerate(self.names)}
    self.fmt = y.dtype
    # Q1: why is there no need for class_to_name?

  def _map_reshape(self, f, arr):
    return np.array([f(elem) for elem in arr.flatten()]).reshape(arr.shape)
    # Q2: why is reshaping necessary?

  def transform(self, y):
    return self._map_reshape(lambda name: self.name_to_class[name], y)

  def fit_transform(self, y):
    self.fit(y)
    return self.transform(y)

  def detransform(self, y_hat):
    return self._map_reshape(lambda idx: self.names[idx], y_hat)

In [3]:
class BaseBayesianClassifier:
  def __init__(self):
    self.encoder = ClassEncoder()

  def _estimate_a_priori(self, y):
    a_priori = np.bincount(y.flatten().astype(int)) / y.size
    # Q3: what does bincount do?
    return np.log(a_priori)
    
  def _fit_params(self, X, y):
    # estimate all needed parameters for given model
    raise NotImplementedError()

  def _predict_log_conditional(self, x, class_idx):
    # predict the log(P(x|G=class_idx)), the log of the conditional probability of x given the class
    # this should depend on the model used
    raise NotImplementedError()

  def fit(self, X, y, a_priori=None):
    
    # first encode the classes
    y = self.encoder.fit_transform(y)

    # if it's needed, estimate a priori probabilities
    self.log_a_priori = self._estimate_a_priori(y) if a_priori is None else np.log(a_priori)

    # check that a_priori has the correct number of classes
    assert len(self.log_a_priori) == len(self.encoder.names), "A priori probabilities do not match number of classes"

    # now that everything else is in place, estimate all needed parameters for given model
    self._fit_params(X, y)
    # Q4: why do we need to do this last, can't we do it first?

  def predict(self, X):
    # this is actually an individual prediction encased in a for-loop
    m_obs = X.shape[1]
    y_hat = np.empty(m_obs, dtype=self.encoder.fmt)
    
    for i in range(m_obs):
      encoded_y_hat_i = self._predict_one(X[:,i].reshape(-1,1))
      y_hat[i] = self.encoder.names[encoded_y_hat_i]

    # return prediction as a row vector (matching y)
    return y_hat.reshape(1,-1)

  def _predict_one(self, x):
    # calculate all log posteriori probabilities (actually, +C)
    log_posteriori = [ log_a_priori_i + self._predict_log_conditional(x, idx) for idx, log_a_priori_i in enumerate(self.log_a_priori) ]

    # return the class that has maximum a posteriori probability
    return np.argmax(log_posteriori)

In [4]:
class QDA(BaseBayesianClassifier):

  def _fit_params(self, X, y):
    # estimate each covariance matrix
    self.inv_covs = [inv(np.cov(X[:,y.flatten()==idx], bias=True)) for idx in range(len(self.log_a_priori))]
    # Q5: why not just X[:,y==idx]?
    # Q6: what does bias=True mean? why not use bias=False?
    self.means = [X[:,y.flatten()==idx].mean(axis=1, keepdims=True) 
                  for idx in range(len(self.log_a_priori))]
    # Q7: what does axis=1 mean? why not axis=0 instead?

  def _predict_log_conditional(self, x, class_idx):
    # predict the log(P(x|G=class_idx)), the log of the conditional probability of x given the class
    # this should depend on the model used
    inv_cov = self.inv_covs[class_idx]
    unbiased_x =  x - self.means[class_idx]
    return 0.5*np.log(det(inv_cov)) -0.5 * unbiased_x.T @ inv_cov @ unbiased_x

In [5]:
class QDA_Original(BaseBayesianClassifier):

  def _fit_params(self, X, y):
    # estimate each covariance matrix
    self.inv_covs = [inv(np.cov(X[:,y.flatten()==idx], bias=True)) for idx in range(len(self.log_a_priori))]
    # Q5: why not just X[:,y==idx]?
    # Q6: what does bias=True mean? why not use bias=False?
    self.means = [X[:,y.flatten()==idx].mean(axis=1, keepdims=True) 
                  for idx in range(len(self.log_a_priori))]
    # Q7: what does axis=1 mean? why not axis=0 instead?

  def _predict_log_conditional(self, x, class_idx):
    # predict the log(P(x|G=class_idx)), the log of the conditional probability of x given the class
    # this should depend on the model used
    inv_cov = self.inv_covs[class_idx]
    unbiased_x =  x - self.means[class_idx]
    return -0.5*np.log(det(inv_cov)) -0.5 * unbiased_x.T @ inv_cov @ unbiased_x

## Código para pruebas

In [6]:
# hiperparámetros
rng_seed = 6543

In [7]:
from sklearn.datasets import load_iris

def get_iris_dataset():
  data = load_iris()
  X_full = data.data
  y_full = np.array([data.target_names[y] for y in data.target.reshape(-1,1)])
  return X_full, y_full

X_full, y_full = get_iris_dataset()

print(f"X: {X_full.shape}, Y:{y_full.shape}")

X: (150, 4), Y:(150, 1)


In [8]:
# peek data matrix
X_full[:5]

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2]])

In [9]:
# peek target vector
y_full[:5]

array([['setosa'],
       ['setosa'],
       ['setosa'],
       ['setosa'],
       ['setosa']], dtype='<U10')

In [10]:
np.unique(y_full)

array(['setosa', 'versicolor', 'virginica'], dtype='<U10')

In [11]:
# preparing data, train - test validation
# 70-30 split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_full, y_full, test_size=0.4, random_state=rng_seed)

# traspose everything because this model format assumes column vectors
train_x = X_train.T
train_y = y_train.T
test_x = X_test.T
test_y = y_test.T

print(train_x.shape, train_y.shape, test_x.shape, test_y.shape)

(4, 90) (1, 90) (4, 60) (1, 60)


In [12]:
qda = QDA()

qda.fit(train_x, train_y)

In [13]:
def accuracy(y_true, y_pred):
  return (y_true == y_pred).mean()

train_acc = accuracy(train_y, qda.predict(train_x))
test_acc = accuracy(test_y, qda.predict(test_x))
print(f"Train (apparent) error is {1-train_acc:.4f} while test error is {1-test_acc:.4f}")

Train (apparent) error is 0.0111 while test error is 0.0167


# Consigna

## Implementación
1. Entrenar un modelo QDA utilizando ahora una *a priori* uniforme ¿Se observan diferencias?¿Por qué?
2. Implementar el modelo LDA, entrenarlo y testearlo contra los mismos sets que QDA ¿Se observan diferencias? ¿Podría decirse que alguno de los dos es notoriamente mejor que el otro?
3. Utilizar otros 2 (dos) valores de *random seed* para obtener distintos splits de train y test, y repetir la comparación del punto anterior ¿Qué se observa?
1. *(Opcional)* En `BaseBayesianClassifier._predict_one` se estima la posteriori de cada clase por separado, a pesar de que la cuenta es siempre la misma (cambiando valores de parámetros, pero no dimensiones). Aprovechando el *broadcasting* de NumPy, se puede reemplazar ese list comprehension por un cálculo *tensorizado* donde tanto medias como varianzas (o inversas) estén "stackeadas" sobre un tercer eje, permitiendo el cálculo en paralelo de todas las clases con un:
`log_posteriori = self.tensor_log_a_priori + self._predict_log_conditionals(x)`. Implementar dicha modificación y mostrar su uso. *Ayuda: los métodos `np.stack` y la documentación del operador [`@`](https://numpy.org/doc/stable/reference/generated/numpy.matmul.html) son de gran utilidad.*

## Preguntas técnicas

Responder las 7 preguntas que se encuentran distribuidas a lo largo del código.

## Preguntas teóricas

1. En LDA se menciona que la función a maximizar puede ser, mediante operaciones, convertida en:
$$
\log{f_j(x)} =  \mu_j^T \Sigma^{-1} (x- \frac{1}{2} \mu_j) + C'
$$
Mostrar los pasos por los cuales se llega a dicha expresión.

2. Explicar, utilizando las respectivas funciones a maximizar, por qué QDA y LDA son "quadratic" y "linear".

3. La implementación de QDA estima la probabilidad condicional utilizando `0.5*np.log(det(inv_cov)) -0.5 * unbiased_x.T @ inv_cov @ unbiased_x` que no es *exactamente* lo descrito en el apartado teórico ¿Cuáles son las diferencias y por qué son expresiones equivalentes?

El espíritu de esta componente práctica es la de establecer un mínimo de trabajo aceptable para su entrega; se invita al alumno a explorar otros aspectos que generen curiosidad, sin sentirse de ninguna manera limitado por la consigna.

## Ejercicio teórico

Sea una red neuronal de dos capas, la primera de 3 neuronas y la segunda de 1 con los parámetros inicializados con los siguientes valores:
$$
w^{(1)} = 
\begin{pmatrix}
0.1 & -0.5 \\
-0.3 & -0.9 \\ 
0.8 & 0.02
\end{pmatrix},
b^{(1)} = \begin{pmatrix}
0.1 \\
0.5 \\
0.8 
\end{pmatrix},
w^{(2)} = 
\begin{pmatrix}
-0.4 & 0.2 & -0.5
\end{pmatrix},
b^{(2)} = 0.7
$$

y donde cada capa calcula su salida vía

$$
y^{(i)} = \sigma (w^{(i)} \cdot x^{(i)}+b^{(i)})
$$

donde $\sigma (z) = \frac{1}{1+e^{-z}}$ es la función sigmoidea .

\\
Dada la observación $x=\begin{pmatrix}
1.8 \\
-3.4
\end{pmatrix}$, $y=5$ y la función de costo $J(\theta)=\frac{1}{2}(\hat{y}_\theta-y)^2$, calcular las derivadas de J respecto de cada parámetro $w^{(1)}$, $w^{(2)}$, $b^{(1)}$, $b^{(2)}$.

# Solucion

## Ejercicio 1

In [14]:
qda1 = QDA()

class_count = len(np.unique(y_train))

qda1.fit(train_x, train_y, a_priori=[1/class_count]*class_count)
train_acc = accuracy(train_y, qda1.predict(train_x))
test_acc = accuracy(test_y, qda1.predict(test_x))
print(f"Train (apparent) error is {1-train_acc:.4f} while test error is {1-test_acc:.4f}")

Train (apparent) error is 0.0222 while test error is 0.0167


Se observan diferencias en el entrenamiento pero no en el test. Si se observan los datos, puntualmente las y, estos no siguen una distribucion uniforme pero se aproximan mucho: las tres clases aparecen un numero similar de veces en el dataset. Si no se asigna una a priori se estimara con la muestra. Entonces usar una uniforme estricta de a priori no es tan diferente a usar la distribucion que se usa de forma automatica. Esto probablemente implica que los dos modelos finalmente son muy parecidos y eso es lo que genera la diferencia en el entrenamiento y obtener el mismo valor en el test. De todas formas con una sola prueba es dificil determinar estrictamente que es lo que esta pasando.

In [15]:
ce = ClassEncoder()

np.bincount(ce.fit_transform(y_train).flatten().astype(int)) / y_train.size

array([0.3       , 0.32222222, 0.37777778])

In [16]:
print(f"QDA INV COVS: \n{qda.inv_covs}")
print(f"QDA LOG A PRIORI: {qda.log_a_priori}")

print(f"QDA UNIFORM INV COVS: \n{qda1.inv_covs}")
print(f"QDA UNIFORM LOG A PRIORI: {qda1.log_a_priori}")

QDA INV COVS: 
[array([[ 22.09817859, -15.69123833,  -3.54800691,   0.58063186],
       [-15.69123833,  19.57946289,   1.25130614,  -9.72088256],
       [ -3.54800691,   1.25130614,  55.34972828,  -9.87952557],
       [  0.58063186,  -9.72088256,  -9.87952557, 106.5198752 ]]), array([[ 10.9780256 ,  -5.88010888, -10.9906851 ,  10.30978873],
       [ -5.88010888,  21.0508669 ,   5.57540582, -23.9370431 ],
       [-10.9906851 ,   5.57540582,  31.49015035, -54.61305166],
       [ 10.30978873, -23.9370431 , -54.61305166, 151.02211857]]), array([[ 10.26219903,  -6.09026899,  -9.10771516,   3.33185775],
       [ -6.09026899,  22.47444984,   0.9369471 , -12.68803238],
       [ -9.10771516,   0.9369471 ,  13.89585808,  -2.5501201 ],
       [  3.33185775, -12.68803238,  -2.5501201 ,  19.38626185]])]
QDA LOG A PRIORI: [-1.2039728  -1.13251384 -0.97344915]
QDA UNIFORM INV COVS: 
[array([[ 22.09817859, -15.69123833,  -3.54800691,   0.58063186],
       [-15.69123833,  19.57946289,   1.25130614,  -9

## Ejercicio 2

In [17]:
class LDA(BaseBayesianClassifier):
    
    def _fit_params(self, X, y):
      # estimate each covariance matrix
      self.inv_covs = [inv(np.cov(X[:,y.flatten()==y.flatten()[0]], bias=True))]
      # Q5: why not just X[:,y==idx]?
      # Q6: what does bias=True mean? why not use bias=False?
      self.means = [X[:,y.flatten()==idx].mean(axis=1, keepdims=True) 
                    for idx in range(len(self.log_a_priori))]
      # Q7: what does axis=1 mean? why not axis=0 instead?

    def _predict_log_conditional(self, x, class_idx):
      # predict the log(P(x|G=class_idx)), the log of the conditional probability of x given the class
      # this should depend on the model used
      inv_cov = self.inv_covs
      return self.means[class_idx].T @ inv_cov @ (x-0.5*self.means[class_idx])

In [18]:
lda = LDA()

lda.fit(train_x, train_y)

train_acc = accuracy(train_y, lda.predict(train_x))
test_acc = accuracy(test_y, lda.predict(test_x))
print(f"Train (apparent) error is {1-train_acc:.4f} while test error is {1-test_acc:.4f}")

Train (apparent) error is 0.0333 while test error is 0.0167


Aunque en test son identicos en entrenamiento QDA es 3 veces mejor que LDA. Dado que en test performan de la misma manera no hay razon para decir que uno es necesariamente mejor que el otro, aunque QDA logra ajustarse a los datos con los que entrena mejor que LDA

## Ejercicio 3

In [19]:
X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(X_full, y_full, test_size=0.4, random_state=42)

# traspose everything because this model format assumes column vectors
train_x_1 = X_train_1.T
train_y_1 = y_train_1.T
test_x_1 = X_test_1.T
test_y_1 = y_test_1.T

print(train_x_1.shape, train_y_1.shape, test_x_1.shape, test_y_1.shape)

(4, 90) (1, 90) (4, 60) (1, 60)


In [20]:
qda.fit(train_x_1, train_y_1)

train_acc = accuracy(train_y_1, lda.predict(train_x_1))
test_acc = accuracy(test_y_1, lda.predict(test_x_1))
print(f"Train (apparent) error is {1-train_acc:.4f} while test error is {1-test_acc:.4f}")

Train (apparent) error is 0.0444 while test error is 0.0000


In [21]:
lda.fit(train_x_1, train_y_1)

train_acc = accuracy(train_y_1, lda.predict(train_x_1))
test_acc = accuracy(test_y_1, lda.predict(test_x_1))
print(f"Train (apparent) error is {1-train_acc:.4f} while test error is {1-test_acc:.4f}")

Train (apparent) error is 0.0222 while test error is 0.0167


Para este split QDA fue perfecto en test, por lo que se lo podria considerar mejor en este escenario, aunque no se logro adaptar al train set de la misma manera que lo logro LDA

In [22]:
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X_full, y_full, test_size=0.4, random_state=36)

# traspose everything because this model format assumes column vectors
train_x_2 = X_train_2.T
train_y_2 = y_train_2.T
test_x_2 = X_test_2.T
test_y_2 = y_test_2.T

print(train_x_2.shape, train_y_2.shape, test_x_2.shape, test_y_2.shape)

(4, 90) (1, 90) (4, 60) (1, 60)


In [23]:
qda.fit(train_x_2, train_y_2)

train_acc = accuracy(train_y_2, lda.predict(train_x_2))
test_acc = accuracy(test_y_2, lda.predict(test_x_2))
print(f"Train (apparent) error is {1-train_acc:.4f} while test error is {1-test_acc:.4f}")

Train (apparent) error is 0.0333 while test error is 0.0000


In [24]:
lda.fit(train_x_2, train_y_2)

train_acc = accuracy(train_y_2, lda.predict(train_x_2))
test_acc = accuracy(test_y_2, lda.predict(test_x_2))
print(f"Train (apparent) error is {1-train_acc:.4f} while test error is {1-test_acc:.4f}")

Train (apparent) error is 0.0222 while test error is 0.0000


En este caso los dos pudieron predecir de forma perfecta en test. QDA se ajusto menos a los valores de entrenamiento. Observando estos 3 experimentos es dificil determinar si alguno de los dos es objetivamente mejor, aunque QDA parece ser un poco mas confiable, dado que en las pruebas de test fue un poco mejor

# Respuestas

## Preguntas tecnicas

1) Porque se puede encontrar en self.names. Si se pide self.names[x] el resultado sera concordante a lo que esta en name_to_class

2) Porque se descompuso la matriz en un arreglo para poder aplicar facilmente la funcion pero es necesario volver a armar una matriz de las mismas dimensiones y con esos valores calculados y no devolver el arreglo que se uso para calcular

3) bincount cuenta la frecuencia de todos los numeros mayores o iguales a 0 dentro de un arreglo. Lo devuelve como un arreglo donde la posicion corresponde al numero al que se le esta calculando la frecuencia. Por ejemplo, si 5 aparece 3 veces en el arreglo a np.bincount(a)[5] == 3

4) No se puede porque el metodo depende de self.log_a_priori. Mientras esa variable no este asignada o ese metodo no deje de depender de esa variable no se puede sacar.

5) Porque Y es una matriz y no un arreglo. Con el flatten se convierte en un arreglo y a partir de eso se puede usarlo para seleccionar los indices de X que se necesitan

6) El parametro bias cambia la forma en la que se calcula la varianza muestral. Si se usa True entonces se calcula usando N y se deberia usar cuando se tiene una muestra chica. Con False se usa (N-1) y se deberia usar cuando la muestra es grande . En este caso usar True sirve, porque se es logico asumir que la muestra con la que se va a entrenar deberia ser relativamente grande

7) axis sirve para determinar como aplicar la transformacion que se esta usando: si se una axis = 0 la funcion se aplica a lo largo de las columnas y si se usa axis = 1 la funcion se aplica a lo largo de las filas. En este caso el resultado es la media de cada una de las features para una clase particular. 

## Preguntas teoricas

1 - Se parte de la funcion 

$$
\log{f_j(x)} =  -\frac{1}{2}\log |\Sigma| - \frac{1}{2} (x-\mu_j)^T \Sigma^{-1} (x- \mu_j) + C
$$ 

$$
C' = C -\frac{1}{2}\log |\Sigma| \Rightarrow \log{f_j(x)} =  - \frac{1}{2} (x-\mu_j)^T \Sigma^{-1} (x- \mu_j) + C
$$

$$
log{f_j(x)} =  - \frac{1}{2} (x-\mu_j)^T \Sigma^{-1} (x- \mu_j) + C
$$

$$
log{f_j(x)} =  - \frac{1}{2} (x^T-\mu_j^T) \Sigma^{-1} (x- \mu_j) + C
$$

$$
log{f_j(x)} =  - \frac{1}{2} (x^T\Sigma^{-1}-\mu_j^T\Sigma^{-1})  (x- \mu_j) + C
$$

$$
log{f_j(x)} =  - \frac{1}{2} (x^T\Sigma^{-1}x - x^T\Sigma^{-1}\mu_j-\mu_j^T\Sigma^{-1}x + \mu_j^T\Sigma^{-1}\mu_j) + C
$$

Como $\Sigma^{-1}$ es Hermitiana (o simetrica si se considera que es real) entonces :

$$
 x^T\Sigma^{-1}\mu_j = \mu_j^T\Sigma^{-1}x 
$$

Entonces:

$$
log{f_j(x)} =  - \frac{1}{2} (x^T\Sigma^{-1}x - 2x^T\Sigma^{-1}\mu_j + \mu_j^T\Sigma^{-1}\mu_j) + C
$$

$$
log{f_j(x)} =  - \frac{1}{2}x^T\Sigma^{-1}x - x^T\Sigma^{-1}\mu_j + - \frac{1}{2}\mu_j^T\Sigma^{-1}\mu_j + C
$$

$$
\log{f_j(x)} =  \mu_j^T \Sigma^{-1} (x- \frac{1}{2} \mu_j) - \frac{1}{2}x^T\Sigma^{-1}x +  C'
$$


Ahora es importante notar que lo relevante de cada $f_{j}(x)$ no es su valor nominal sino su valor relativo con los otros $f_{j}(x)$. 

Supongase que como en este caso se tienen tres clases (1, 2, 3) y una observacion $x_{1}$. Entonces se calcularian $log f_{1}(x_{1})$, $log f_{2}(x_{1})$ y $log f_{3}(x_{1})$ y se buscaria el maximo de los tres.

Si se observa la ultima expresion de $\log{f_j(x)}$ se puede observar que hay un termino que depende exclusivamente de x. Eso implica que ese termino para $log f_{1}(x_{1})$, $log f_{2}(x_{1})$ y $log f_{3}(x_{1})$
valdra lo mismo. Y como vale lo mismo para los tres no afecta a cual de los tres es finalmente el maximo. Por ende se puede descartar sin afectar el resultado final.

Entonces 

$$
\log{f_j(x)} =  \mu_j^T \Sigma^{-1} (x- \frac{1}{2} \mu_j) +  C'
$$


2 - Se puede ver que en el QDA hay un producto de $x^T$ con x, lo que hace que aparezca $x^2$, en el LDA solo aparece x, por lo tanto en uno es lineal y en el otro es cuadrático con x. 

![alt text for screen readers](images/qda-lda.jpeg "Text to show on mouseover")


3 -  
Formula implementada:  

$$\frac{1}{2} log|\Sigma _j|^{-1}-\frac{1}{2}(x-\mu _j)^T \Sigma _j ^{-1}(x-\mu _j)$$

Formula teórica: 

$$-\frac{1}{2} log|\Sigma _j|-\frac{1}{2}(x-\mu _j)^T \Sigma _j ^{-1}(x-\mu _j)+C$$

Se debe demostrar que: 

$$\frac{1}{2} log|\Sigma _j|^{-1}-\frac{1}{2}(x-\mu _j)^T \Sigma _j ^{-1}(x-\mu _j)=-\frac{1}{2} log|\Sigma _j|-\frac{1}{2}(x-\mu _j)^T \Sigma _j ^{-1}(x-\mu _j)+C$$

Sumando en ambos lados $\frac{1}{2}(x-\mu _j)^T \Sigma _j ^{-1}(x-\mu _j)$: 

$$\frac{1}{2} log|\Sigma _j|^{-1}=-\frac{1}{2} log|\Sigma _j|+C$$


Por propiedad de logaritmo, $log|\Sigma _j|^{-1}=-log|\Sigma _j|$, entonces:

$$0=C$$ 

Por lo tanto se demuestra que la formula implementada es la misma que la formula teórica utilizando C=0.

## Ejercicio teorico

Para este ejercicio se tiene el siguiente grafo que representa las diferentes capas de la red neuronal

![alt text for screen readers](images/grafo2.png "Text to show on mouseover")

El primer paso es calcular $Z1$, $Y1$, $Z2$, $Ypred$ y $J$.

$$

Z1 = \begin{pmatrix}
1.98 \\
3.02 \\ 
2.172 
\end{pmatrix} ,

Y1 = \begin{pmatrix}
0.8786 \\
0.9534 \\ 
0.8977 
\end{pmatrix} ,

Z2 = 0.0903 ,

Ypred = 0.5225 

$$

Se calcula tambien la derivada de $\sigma$
$$
\sigma'(x) = \frac{-e^x}{2 e^x + (e^x)^2 + 1}
$$

Ahora es necesario definir las derivadas que se buscan calcular. Estas se pueden definir de la siguiente manera usando la regla de la cadena:

$$

\frac{dJ}{dw_{2}} = \frac{dJ}{dY_{pred}}  \frac{dY_{pred}}{dz_{2}} \frac{dz_{2}}{dw_{2}}

$$

$$

\frac{dJ}{db_{2}} = \frac{dJ}{dY_{pred}}  \frac{dY_{pred}}{dz_{2}} \frac{dz_{2}}{db_{2}} 

$$

$$

\frac{dJ}{dw_{1}} = \frac{dJ}{dY_{pred}}  \frac{dY_{pred}}{dz_{2}} \frac{dz_{2}}{dY1} \frac{dY1}{dz_{1}} \frac{dz_{1}}{dw_{1}}

$$

$$

\frac{dJ}{db_{1}} = \frac{dJ}{dY_{pred}}  \frac{dY_{pred}}{dz_{2}} \frac{dz_{2}}{dY1} \frac{dY1}{dz_{1}} \frac{dz_{1}}{db_{1}}

$$

Para $\frac{dJ}{dw_{2}}$

$$
\frac{dJ}{dY_{pred}} = (Y_{pred} - Y) ,

\frac{dY_{pred}}{dz_{2}} = \sigma'(z2),

\frac{dz_{2}}{dw_{2}} = Y1

$$

$$

\frac{dJ}{dw_{2}} = \frac{dJ}{dY_{pred}}  \frac{dY_{pred}}{dz_{2}} \frac{dz_{2}}{dw_{2}}

\Rightarrow 

\frac{dJ}{dw_{2}} = (Y_{pred} - Y) \sigma'(z2) Y1^T

$$


Reemplazando con los valores conocidos

$$

\frac{dJ}{dw_{2}} = (0.5225 - 5) \sigma'(0.0903) \begin{pmatrix}
0.8786 \\
0.9534 \\ 
0.8977 
\end{pmatrix} ^T = \begin{pmatrix}
-0.98155 \\
-1.06509 \\ 
-1.00280 
\end{pmatrix}^T

$$

Para $\frac{dJ}{db_{2}}$

$$

\frac{dz_{2}}{db_{2}} = 1

$$

$$

\frac{dJ}{db_{2}} = \frac{dJ}{dY_{pred}}  \frac{dY_{pred}}{dz_{2}} \frac{dz_{2}}{db_{2}}

\Rightarrow 

\frac{dJ}{db_{2}} = (Y_{pred} - Y) \sigma'(z2) 1

$$

Reemplazando con los valores conocidos

$$

\frac{dJ}{db_{2}} = (0.5225 - 5) \sigma'(0.0903) = -1.11707

$$

Para $\frac{dJ}{dw_{1}}$

$$

\frac{dY_{pred}}{dz_{2}} = \sigma'(z2),

\frac{dz_{2}}{dY1} = w_{2},

\frac{dY1}{dz_{1}} = \sigma'(z1),

\frac{dz_{1}}{dw_{1}} = X1

$$

$$

\frac{dJ}{dw_{1}} = \frac{dJ}{dY_{pred}}  \frac{dY_{pred}}{dz_{2}} \frac{dz_{2}}{dY1} \frac{dY1}{dz_{1}} \frac{dz_{1}}{dw_{1}}

\Rightarrow 

\frac{dJ}{dw_{1}} = (Y_{pred} - Y)  \sigma'(z2) w_{2}^T  \sigma'(z1) X1^T

$$

Reemplazando con los valores conocidos

$$

\frac{dJ}{dw_{1}} = (0.5225 - 5) \sigma'(0.0903) \begin{pmatrix}
-0.4 & 0.2 & -0.5
\end{pmatrix}^T \odot \sigma'(\begin{pmatrix}
1.98 \\
3.02 \\ 
2.172 
\end{pmatrix}) \begin{pmatrix}
1.8 \\
-3.4
\end{pmatrix} ^T = \begin{pmatrix}
0.0857 & -0.1619 \\
- 0.0178 & 0.033 \\
0.0923 & -0.1743 
\end{pmatrix}

$$

Para $\frac{dJ}{db_{1}}$

$$

\frac{dz_{1}}{db_{1}} = 1

$$

$$

\frac{dJ}{db_{1}} = \frac{dJ}{dY_{pred}}  \frac{dY_{pred}}{dz_{2}} \frac{dz_{2}}{dY1} \frac{dY1}{dz_{1}} \frac{dz_{1}}{db_{1}}
$$

Reemplazando con los valores conocidos

$$

\frac{dJ}{db_{1}} = ((0.5225 - 5) \sigma'(0.0903) \begin{pmatrix}
-0.4 & 0.2 & -0.5
\end{pmatrix}^T \odot \sigma'(\begin{pmatrix}
1.98 \\
3.02 \\ 
2.172 
\end{pmatrix}) =\begin{pmatrix}
0.0476 \\
-0.0099 \\ 
0.0512 
\end{pmatrix}

$$